In [3]:
# Importing Basic Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [4]:
# Importing Required Sklearn and Keras Functions
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
# Setting the Random Seed
seed = 2
np.random.seed(seed)

In [6]:
# Load the training Data
train = pd.read_csv("../input/train.csv")
print(train.shape)

(42000, 785)


In [7]:
# Creating X and y to fit on the model
y = train["label"]
X = train.drop("label", axis = 1) 
# Get a look at the Images
print(y.value_counts().to_dict())
y = to_categorical(y, num_classes = 10)



{1: 4684, 7: 4401, 3: 4351, 9: 4188, 2: 4177, 6: 4137, 0: 4132, 4: 4072, 8: 4063, 5: 3795}


In [8]:
# Normalizing the Data
X = X / 255.0
# Reshaping the Grayscale Images to be fit into the CNN
X = X.values.reshape(-1,28,28,1)


In [9]:
# Splitting into Training and Validation Sets
train_index, valid_index = ShuffleSplit(n_splits=1, train_size=0.9, test_size=None, random_state=seed).split(X).__next__()
train_x = X[train_index]
train_y = y[train_index]
valid_x = X[valid_index]
valid_y = y[valid_index]
print(train_x.shape, valid_x.shape)

(37800, 28, 28, 1) (4200, 28, 28, 1)


In [10]:
# Creating the Model
model = Sequential()


In [12]:
# First Block with 2 Convultional layers, 1 maxpool layer and with a dropout 
# Valid Padding
model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'Valid', activation ='relu', input_shape = (28,28,1)))
# Same padding
model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same',  activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [13]:
# Second block with 2 Convolutional layers, 1 maxpool layer and with a droput
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))


In [14]:
# Third and final Block with a flatten layer and softmax output layer with a dropout too 
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))


In [15]:
# using an Adam Optimizer
model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-3), metrics=["accuracy"])
# To Reduce Learning Rate once Plateau is reached accuracy wise
annealer = ReduceLROnPlateau(monitor='val_acc', patience=2, verbose=1, factor=0.5, min_lr=0.00001)

In [16]:
# Setting Epoch and batch Size
epochs = 20
batch_size = 64

In [17]:
# Pre Processing the Data to be fit into the model 
datagen = ImageDataGenerator(featurewise_center=False, samplewise_center=False, featurewise_std_normalization=False,
        samplewise_std_normalization=False, zca_whitening=False, rotation_range=10, zoom_range = 0.1, width_shift_range=0.1,
        height_shift_range=0.1, horizontal_flip=False, vertical_flip=False)



In [18]:
# Fitting on the Model
model.fit_generator(datagen.flow(train_x,train_y,batch_size=batch_size), epochs=epochs, validation_data=(valid_x[:600,:],valid_y[:600,:]),
                              verbose = 2, steps_per_epoch=train_x.shape[0]//batch_size, callbacks=[annealer])


Epoch 1/20
 - 124s - loss: 0.4818 - acc: 0.8402 - val_loss: 0.0648 - val_acc: 0.9700
Epoch 2/20
 - 123s - loss: 0.1250 - acc: 0.9647 - val_loss: 0.0405 - val_acc: 0.9900
Epoch 3/20
 - 126s - loss: 0.0936 - acc: 0.9731 - val_loss: 0.0306 - val_acc: 0.9917
Epoch 4/20
 - 122s - loss: 0.0788 - acc: 0.9775 - val_loss: 0.0320 - val_acc: 0.9917
Epoch 5/20
 - 123s - loss: 0.0711 - acc: 0.9803 - val_loss: 0.0332 - val_acc: 0.9900
Epoch 6/20
 - 124s - loss: 0.0652 - acc: 0.9814 - val_loss: 0.0312 - val_acc: 0.9950
Epoch 7/20
 - 119s - loss: 0.0609 - acc: 0.9833 - val_loss: 0.0370 - val_acc: 0.9917
Epoch 8/20
 - 121s - loss: 0.0587 - acc: 0.9839 - val_loss: 0.0206 - val_acc: 0.9950
Epoch 9/20
 - 122s - loss: 0.0536 - acc: 0.9844 - val_loss: 0.0238 - val_acc: 0.9950

Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 10/20
 - 122s - loss: 0.0396 - acc: 0.9886 - val_loss: 0.0106 - val_acc: 0.9967
Epoch 11/20
 - 121s - loss: 0.0336 - acc: 0.9906 - val_loss: 0.0099 

In [19]:
# Predictions
# On Training Set
tr_ps =  model.predict(train_x)
tr_p = np.max(tr_ps, axis=1)
tr_pl = np.argmax(tr_ps, axis=1)
# On Validation Set 
vl_ps = model.predict(valid_x)
vl_p = np.max(vl_ps, axis=1)
vl_pl = np.argmax(vl_ps, axis=1)

In [20]:
# Wrongly Classified Images
# On Training
tr_y = np.argmax(train_y, axis=1)
tr_x_err = train_x[tr_y != tr_pl]
tr_y_err = tr_y[tr_y != tr_pl]
tr_pl_err = tr_pl[tr_y != tr_pl]
# On Validation
vl_y = np.argmax(valid_y, axis=1)
vl_x_err = valid_x[vl_y != vl_pl]
vl_y_err = vl_y[vl_y != vl_pl]
vl_pl_err = vl_pl[vl_y != vl_pl]
# To See the wrongly classified images for Feedback Purposes
'''
for i in range(tr_y_err.shape[0]):
    plt.imsave(fname='train_err_%d_%d_%d.csv'%(tr_y_err[i], tr_pl_err[i], i), arr=tr_x_err[i].reshape(28,28),format='png')
for i in range(vl_y_err.shape[0]):
    plt.imsave(fname='valid_err_%d_%d_%d.csv'%(vl_y_err[i], vl_pl_err[i], i), arr=vl_x_err[i].reshape(28,28),format='png')
'''

"\nfor i in range(tr_y_err.shape[0]):\n    plt.imsave(fname='train_err_%d_%d_%d.csv'%(tr_y_err[i], tr_pl_err[i], i), arr=tr_x_err[i].reshape(28,28),format='png')\nfor i in range(vl_y_err.shape[0]):\n    plt.imsave(fname='valid_err_%d_%d_%d.csv'%(vl_y_err[i], vl_pl_err[i], i), arr=vl_x_err[i].reshape(28,28),format='png')\n"

In [21]:
print('Base model scores:')
valid_loss, valid_acc = model.evaluate(valid_x, valid_y, verbose=0)
print("model valid loss: {0:.4f}, valid accuracy: {1:.4f}".format(valid_loss, valid_acc))


Base model scores:
model valid loss: 0.0202, valid accuracy: 0.9950


In [22]:
# Confusion Matrix on the validation set
valid_p = np.argmax(model.predict(valid_x), axis=1)
target = np.argmax(valid_y, axis=1)
cm = confusion_matrix(target, valid_p)
print(cm)

[[410   0   0   0   0   0   1   0   0   0]
 [  0 482   0   0   0   0   0   2   1   0]
 [  0   0 402   1   0   0   0   0   0   0]
 [  0   0   0 417   0   0   0   0   1   0]
 [  0   0   0   0 458   0   1   0   0   2]
 [  0   0   0   2   0 367   1   0   2   0]
 [  0   0   0   0   1   0 410   0   2   0]
 [  0   0   0   0   0   0   0 445   0   1]
 [  0   0   0   0   0   0   0   0 381   1]
 [  0   0   0   0   1   0   0   1   0 407]]


In [23]:
# Now on to the predictions on the Test Set
test = pd.read_csv("../input/test.csv")
print(test.shape)

(28000, 784)


In [24]:
# Normalizing and Re-Shaping the Test Data to be fit into the CNN we made
test = test / 255.0
test = test.values.reshape(-1,28,28,1)
# Predictions on the Test Set
p = np.argmax(model.predict(test), axis = 1)

In [25]:
# Creating the Submission File
submission = pd.DataFrame(pd.Series(range(1, p.shape[0]+1), name='ImageId'))
submission['Label'] = p
submission.to_csv("CNN.csv", index=False)